In [1]:
import re
import spacy
from transformers import pipeline

In [2]:
# Load the SpaCy English model for lemmatization
nlp = spacy.load('en_core_web_sm')

# Load a pretrained model for text classification
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
def preprocess_and_predict(text: str):
    # Function to remove punctuation
    def remove_punctuation(text: str) -> str:
        return re.sub(r'[^\w\s]+', '', text)

    # Function to remove URLs
    def remove_url(text: str) -> str:
        return re.sub(r'http\S+|www.\S+', '', text)

    # Sample stop words list
    stop_words = set([
        "i", "me", "my", "myself", "we", "our", "ours", "ourselves",
        "you", "your", "yours", "yourself", "yourselves", "he",
        "him", "his", "himself", "she", "her", "hers", "herself",
        "it", "its", "itself", "they", "them", "their", "theirs",
        "themselves", "what", "which", "who", "whom", "this",
        "that", "these", "those", "am", "is", "are", "was", "were",
        "be", "been", "being", "have", "has", "had", "having",
        "do", "does", "did", "doing", "a", "an", "the", "and",
        "but", "if", "or", "because", "as", "until", "while",
        "of", "at", "by", "for", "with", "about", "against",
        "between", "into", "through", "during", "before", "after",
        "above", "below", "to", "from", "up", "down", "in",
        "out", "on", "off", "over", "under", "again", "further",
        "then", "once", "here", "there", "when", "where", "why",
        "how", "all", "any", "both", "each", "few", "more",
        "most", "other", "some", "such", "no", "nor", "not",
        "only", "own", "same", "so", "than", "too", "very",
        "s", "t", "can", "will", "just", "don", "should",
        "now"
    ])

    # Preprocessing steps
    text = remove_punctuation(text)           # Remove punctuation
    text = remove_url(text)                    # Remove URLs
    text = ' '.join([word for word in text.lower().split() if word not in stop_words])  # Remove stop words

    # Lemmatize the preprocessed text
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])

    # Make prediction using the pretrained model
    prediction = classifier(lemmatized_text)

    return lemmatized_text, prediction

In [14]:
if __name__ == "__main__":
    text_input = """Elon Musk is a visionary entrepreneur known for his role in revolutionizing transportation and space exploration.
     As the CEO of SpaceX, he has spearheaded numerous successful rocket launches, aiming to make space travel affordable and accessible.
     In addition to SpaceX, Musk is the driving force behind Tesla, Inc., where he has pushed the boundaries of electric vehicle technology.
     Under his leadership, Tesla has become a leading manufacturer of electric cars, focusing on sustainability and innovation.
     Musk's ambitious plans also include the development of the Hyperloop, a high-speed transportation system, and Neuralink, a company dedicated to advancing brain-computer interface technology.
     With a relentless pursuit of progress and an unwavering belief in the potential of technology, Musk continues to shape the future of multiple industries.."""

    lemmatized_text, prediction = preprocess_and_predict(text_input)

    print(f"Lemmatized Text: {lemmatized_text}")
    print(f"Prediction: {prediction}\n")

Lemmatized Text: elon musk visionary entrepreneur know role revolutionize transportation space exploration ceo spacex spearhead numerous successful rocket launch aim make space travel affordable accessible addition spacex musk drive force behind tesla inc push boundary electric vehicle technology leadership tesla become lead manufacturer electric car focus sustainability innovation musk ambitious plan also include development hyperloop highspeed transportation system neuralink company dedicate advance braincomputer interface technology relentless pursuit progress unwavere belief potential technology musk continue shape future multiple industry
Prediction: [{'label': 'POSITIVE', 'score': 0.9912647604942322}]

